## Carga modulo comun

In [1]:
import sys
import os
sys.path.append(os.path.abspath('../../common'))

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from purrfect.dataset import EPKADataset, load_partition,save_partition, create_train_valid_loaders
from torch.utils.data import DataLoader

from purrfect.training import train_model,train_validate
import torch.optim as optim

from purrfect.metrics import MetricAccumulator
from purrfect.active_learning import create_new_partition,create_next_partitions, test_model

from sklearn.model_selection import train_test_split

In [3]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Definición modelo

In [4]:
# Define the double convolution block
class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(DoubleConv, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.conv(x)

# Define the U-Net model
class UNet(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(UNet, self).__init__()

        # Encoder
        self.encoder1 = DoubleConv(in_channels, 64)
        self.encoder2 = DoubleConv(64, 128)
        self.encoder3 = DoubleConv(128, 256)
        self.encoder4 = DoubleConv(256, 512)

        # Bottleneck
        self.bottleneck = DoubleConv(512, 1024)

        # Decoder
        self.upconv4 = nn.ConvTranspose2d(1024, 512, kernel_size=2, stride=2)
        self.decoder4 = DoubleConv(1024, 512)
        self.upconv3 = nn.ConvTranspose2d(512, 256, kernel_size=2, stride=2)
        self.decoder3 = DoubleConv(512, 256)
        self.upconv2 = nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2)
        self.decoder2 = DoubleConv(256, 128)
        self.upconv1 = nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2)
        self.decoder1 = DoubleConv(128, 64)

        # Final output layer
        self.final_conv = nn.Conv2d(64, out_channels, kernel_size=1)

    def forward(self, x):
        # Encoder
        e1 = self.encoder1(x)
        e2 = self.encoder2(F.max_pool2d(e1, 2))
        e3 = self.encoder3(F.max_pool2d(e2, 2))
        e4 = self.encoder4(F.max_pool2d(e3, 2))

        # Bottleneck
        b = self.bottleneck(F.max_pool2d(e4, 2))

        # Decoder with skip connections
        d4 = self.upconv4(b)
        d4 = torch.cat((e4, d4), dim=1)
        d4 = self.decoder4(d4)

        d3 = self.upconv3(d4)
        d3 = torch.cat((e3, d3), dim=1)
        d3 = self.decoder3(d3)

        d2 = self.upconv2(d3)
        d2 = torch.cat((e2, d2), dim=1)
        d2 = self.decoder2(d2)

        d1 = self.upconv1(d2)
        d1 = torch.cat((e1, d1), dim=1)
        d1 = self.decoder1(d1)

        # Final output
        out = self.final_conv(d1)
        return out

## Entrenamiento

In [5]:
#Creacion de particiones train y valid
init_partition = load_partition("partition_1.json")
train_partition, val_partition = train_test_split(init_partition, test_size=0.2, random_state=42)
save_partition("partition_1_train.json","partitions",train_partition)
save_partition("partition_1_val.json","partitions",val_partition)

## Entrenamiento

In [8]:
#Define model
model = UNet( 3, 1)
model = model.to(DEVICE)
# Define Loss
criterion = torch.nn.MSELoss()
current_partition = 1

In [9]:
train_loader, val_loader = create_train_valid_loaders(
    f"partition_{current_partition}_train.json",
    f"partition_{current_partition}_val.json",
    "partitions",
    batch_size=16,
)

In [10]:
best_model_path = os.path.join(
    "models", f"best_model_partition_{current_partition}.pth"
)
last_checkpoint_path = os.path.join(
    "models", f"last_checkpoint_partition_{current_partition}.pth"
)

In [10]:
optimizer = optim.Adam(model.parameters(), lr=0.00001)
train_model(
    model,
    train_loader,
    val_loader,
    best_model_path,
    last_checkpoint_path,
    criterion,
    optimizer,
    num_epochs=50,
    device=DEVICE,
    early_stopping_patience=3,
)

Epoch [1/50]


Train Epoch 1: Loss: 0.046101 WMAPE: 53.6817 DICE: 0.5466 DPEAKS: 232.8555


Validate Epoch 1: Loss: 0.011560 WMAPE: 27.8533 DICE: 0.5439 DPEAKS: 232.8433
Saving best model
Epoch [2/50]


Train Epoch 2: Loss: 0.005182 WMAPE: 16.4709 DICE: 0.5459 DPEAKS: 229.6230


Validate Epoch 2: Loss: 0.003286 WMAPE: 12.6127 DICE: 0.5440 DPEAKS: 232.5486
Saving best model
Epoch [3/50]


...

Validate Epoch 21: Loss: 0.000597 WMAPE: 5.2411 DICE: 0.3013 DPEAKS: 141.7812
Epoch [22/50]


Train Epoch 22: Loss: 0.000368 WMAPE: 4.0896 DICE: 0.2461 DPEAKS: 102.1929


Validate Epoch 22: Loss: 0.000600 WMAPE: 5.2793 DICE: 0.3057 DPEAKS: 139.6808
Epoch [23/50]


Train Epoch 23: Loss: 0.000351 WMAPE: 4.0392 DICE: 0.2398 DPEAKS: 100.1441


Validate Epoch 23: Loss: 0.000619 WMAPE: 4.9397 DICE: 0.3046 DPEAKS: 141.2727
early stopping: 3 epochs without improvement
Training complete.


In [11]:
#Cargar mejor modelo de la particion actual
model.load_state_dict(torch.load(best_model_path,weights_only=True))

<All keys matched successfully>

In [12]:
test_model(model,criterion,device=DEVICE,batch_size=16)

Validate Epoch test: Loss: 0.000599 WMAPE: 5.0596 DICE: 0.3006 DPEAKS: 135.7086


In [13]:
create_next_partitions(current_partition,model,criterion,device=DEVICE)

Validate Epoch partition_2_train: Loss: 0.000598 WMAPE: 5.0639 DICE: 0.3015 DPEAKS: 137.5044


Validate Epoch partition_2_val: Loss: 0.000592 WMAPE: 5.0809 DICE: 0.3022 DPEAKS: 130.9902


In [14]:
current_partition+=1

In [15]:
train_loader, val_loader = create_train_valid_loaders(
    f"partition_{current_partition}_train.json",
    f"partition_{current_partition}_val.json",
    "partitions",
    batch_size=16,
)
best_model_path = os.path.join(
    "models", f"best_model_partition_{current_partition}.pth"
)
last_checkpoint_path = os.path.join(
    "models", f"last_checkpoint_partition_{current_partition}.pth"
)
optimizer = optim.Adam(model.parameters(), lr=0.00001)
train_model(
    model,
    train_loader,
    val_loader,
    best_model_path,
    last_checkpoint_path,
    criterion,
    optimizer,
    num_epochs=50,
    device=DEVICE,
    early_stopping_patience=3,
)

Epoch [1/50]


Train Epoch 1:   0%|          | 0/307 [00:00<?, ?it/s]

Train Epoch 1: Loss: 0.000481 WMAPE: 4.7386 DICE: 0.2865 DPEAKS: 141.2528


Validate Epoch 1: Loss: 0.000532 WMAPE: 5.2362 DICE: 0.3004 DPEAKS: 148.1830
Saving best model
Epoch [2/50]


Train Epoch 2: Loss: 0.000422 WMAPE: 4.6009 DICE: 0.2741 DPEAKS: 135.1597


Validate Epoch 2: Loss: 0.000556 WMAPE: 5.0526 DICE: 0.3027 DPEAKS: 149.2639
Epoch [3/50]


Train Epoch 3: Loss: 0.000391 WMAPE: 4.5341 DICE: 0.2648 DPEAKS: 129.2787


Validate Epoch 3: Loss: 0.000541 WMAPE: 4.9426 DICE: 0.3011 DPEAKS: 149.4330
Epoch [4/50]


Train Epoch 4: Loss: 0.000370 WMAPE: 4.5166 DICE: 0.2570 DPEAKS: 121.2822


Validate Epoch 4: Loss: 0.000545 WMAPE: 5.1304 DICE: 0.2990 DPEAKS: 149.3047
early stopping: 3 epochs without improvement
Training complete.


In [16]:
#Cargar mejor modelo de la particion actual
model.load_state_dict(torch.load(best_model_path,weights_only=True))
test_model(model,criterion,device=DEVICE,batch_size=16)

Validate Epoch test: Loss: 0.000550 WMAPE: 4.6976 DICE: 0.2913 DPEAKS: 130.4800
